# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования необходимы для учёта при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/datasets/data.csv')
data.to_csv('data.csv', index=False)

In [ ]:
clients = pd.read_csv('/datasets/data.csv')

Изучим данные в датасете: начало, конец и  общую информацию о датасете

In [ ]:
#clients.head(15)
#clients.tail(15)
clients.info()

**Вывод: Имеем 21525 строк, а также 12 столбцов. Основные проблемы, которые я заметил:**

*- названия столбцов(могут давать неправильное представление о содержащихся в них данных или быть вовсе непонятными)*

*- пропуски в days_employed и total_income*

*- отрицательные значения в days_employed*

*- некорректные типы данных — в days_employed и в total_income плавающая точка(могут возникнуть проблемы при подсчётах конкретных значений)*

*- некорректный регистр в education*

## Шаг 2. Предобработка данных

**Для начала разберёмся с проблемой, которая выглядят как незначительные в рамках проекта, но может иметь последствия в будущем, а именно:**

Отдельно выведем названия и значения столбцов, чтобы в последующих шагах было понятней, почему мы обращаемся к той или иной части датасета:

- children — количество детей в семье
- days_employed — трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — образование клиента
- education_id — идентификатор образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — доход в месяц
- purpose — цель получения кредита

P.S. - Можно было бы некоторые столбцы переименовать. Но не факт, что более понятные лично мне названия, будут также понятны заказчику. К тому же, это может вызвать неудобства у коллег, которые будут использовать данный проект в своей работе(если они, например, будут отдельно рассматривать моё исследование и изначальный датасет)

**Так же хочу сразу прояснить ситуацию с, как мне кажется, наиболее проблематичным столбцом:**

**days_employed** - содержит отрицательные значения; имеет пропуски; имеет некорректные данные(Мало того, что записан в днях(а не в годах например), так ещё и имеет значения противоречащие законам логики и анатомии(некоторые люди работают с пелёнок). 
Этот столбец и столбец *total_income* имеют одинаковое количество строк, что может свидетельствовать об их взаимосвязанности, но трудно сказать о какой именно связи идёт речь.
Тем не менее, я считаю, что данный столбец не потребуетя для выявления зависимостей между возвратом кредита в срок и наличием детей/семейным положением/уровнем дохода за месяц/целями кредита, поэтому считаю целесообразным его УДАЛИТЬ, заранее предупредив коллег.*

In [ ]:
del clients['days_employed']

In [ ]:
clients.info()

### Обработка пропусков

Далее обработаем пропуски в доходах за месяц. Пропуски в данном столбце могли появиться по разным причинам:

*1. Человек ещё не работал или работал только неофициально*

*2. Человек не указал свой ежемесячный доход*

*3. Произошла ошибка при выгрузке данных*

Заменим пропущенные значения на среднее арифметическое.
И проверим наличие пропусков методом *info()* теперь

In [ ]:
#Сначала сделаем функцию для замены пропущенных значений по категориям при помощи цикла for
#При чём сделаем её универсальной, чтобы можно было применить эту функцию к разным датафреймам
def no_null(df, category, target):
    for i in df[category].unique():
        df.loc[(df[target].isna()) & (df[category] == i), target] = \
        df.loc[df[category] == i, target].median()
    return df

In [ ]:
# Код ревьюера

display(clients[clients['total_income'].isna()]['income_type'].value_counts(normalize=True))
display(clients[clients['total_income'].notna()]['income_type'].value_counts(normalize=True))

In [ ]:
#Применим функцию для наших значений
clients = no_null(clients, 'income_type', 'total_income')

Также после замены подсчитаем суммарное количество пропусков, на случай если мы что то упустили

In [ ]:
clients.isnull().sum()

Предположив, что уровень дохода зависит от типа занятости, заменили пропуски в *total_income* по медианному значению среди людей с таким же типом занятости. Для этого написали функцию с циклом for и применили данную функцию к нашему датафрейму. 

### Замена типа данных

Снова изучим датасет, на предмет удобства для дальнейших расчётов.

Видим, что данные в столбце *total_income* имеют тип *float64*

In [ ]:
clients.info()
print(clients['total_income'])

Поменяем тип данных в столбце *total_income* на *int64* методом *astype()*

*Мы не станем использовать метод to_numeric() потому что при переводе все числа будут иметь тип данных float. А нам необходим именно тип данных int, т.к. нам нужны целочисленный значения*

In [ ]:
# Код ревьюера

s = pd.Series([1.2,3,5.2])

print(s.dtype)

s = pd.to_numeric(s.round(), downcast='integer')

print(s.dtype)

In [ ]:
clients['total_income'] = clients['total_income'].astype('int')
print(clients['total_income'])

**Вывод**

Мы поменяем тип данных в столбце *total_income*, чтобы привести значения к целочисленному формату. Это поможет представить дальнейшие расчёты в более удобном виде

### Обработка дубликатов

In [ ]:
clients.duplicated().sum()

In [ ]:
clients['total_income'].value_counts()

Ранее мы выявили проблему в столбце *education*. Проблема заключалась в некорректном регистре. Поэтому сейчас мы ещё раз увидим проблему, подсчитав различные значения в списке

In [ ]:
from collections import Counter

Counter(clients['education'])

Приведем в единый формат данный столбец, убрав дубликаты, полученные из-за некорректного регистра.

И ещё раз посчитаем различные значения при помощи коллекции Counter, реализующей словарь для подсчёта колличества неизменяемых объектов

In [ ]:
clients['education'] = clients['education'].str.lower()

Counter(clients['education'])

**Таким образом, мы решили все проблемы, выявленные самостоятельно на начальном этапе изучения общей информации. Далее мы будем действовать исходя из тех вопросов, которые перед нами ставит заказчик. А именно: выявление зависимостей между возвратом кредита в срок и наличием детей, семейным положением, уровнем дохода и целями кредита.**

Для выявления зависимостей нам необходимо проверить на дубликаты следующие столбцы: *children*, *family_status*, *total_income* и *purpose*

In [ ]:
Counter(clients['children'])

Видим, что среди значений столбца *children* есть неадекватные. А именно "-1" и "20". 

Мы не знаем чем обусловлено значение "-1". Возможно при вводе случайно поставили "-" перед единицей. Возможно "-" использовали при вводе в качетсве тире. Будем исходить из того, что хотели написать "1" и поменяем соответсвенно на единицу.

Что касается значения в 20 детей, то тут можно предположить что произошла опечатка при попытка написать "0" или "2". Т.к. это могло быть и 0 детей и 2, считаю логичным заменить 20 медианным значением

In [ ]:
clients['children'].value_counts() #было

In [ ]:
clients['children'] = clients['children'].replace(-1, 1)
children_median = clients.loc[clients.loc[:, 'children'] != 20]['children'].median()
clients['children'] = clients['children'].replace(20, children_median)

In [ ]:
clients['children'].value_counts() #стало

**Вернёмся к проверке на дубликаты**

In [ ]:
Counter(clients['family_status'])

In [ ]:
Counter(clients['purpose'])

Как видим, столбцы *family_status* и *purpos* прошли проверку на дубликаты без проблем. Теперь осталось проверить столбец *total_income*

In [ ]:
Counter(clients['total_income'])

Результат в не сильно удобном для чтения виде, однако можно предположить, что дубликаты в столбце *total_income* есть. Проверим так ли это найдя сумму всех дубликатов в датасете, и сравнив значения в столбце *total_income* до и после их удаления

In [ ]:
clients.duplicated().sum() #Всего До

In [ ]:
print(clients['total_income'].value_counts()) #В столбце До

In [ ]:
clients = clients.drop_duplicates()

In [ ]:
clients.duplicated().sum() #Всего После

Для поиска использовали метод *duplicated()*. В сочетании с методом *sum()* он возвращает количество дубликатов.

Затем мы удалили их при помощи метода *drop_duplicates()*

In [ ]:
print(clients['total_income'].value_counts()) #В столбце После

Это также можно увидеть выведя на экран общую информацию по датасету

In [ ]:
clients.info()

**Вывод**

Мы выявили и устранили ошибки в столбцах *education* и *children*, а также обнаружили дубликаты в датасете. Причиной появления дубликатов может быть обычное задвоение данных. После обнаружения мы удалили все дубликаты, чтобы они в дальнейшем повлияли на наши оценки

### Лемматизация

In [ ]:
from pymystem3 import Mystem
m = Mystem()

def lemmatization(i):
    lemmas = m.lemmatize(i)                     
    return lemmas

clients['purpose_lemma'] = clients['purpose'].apply(lemmatization)
clients.head(15)

**Вывод**

При помощи функции *lemmatization* сделали лемматизацию и вывели результаты в отдельный столбец *purpose_lemma*

### Категоризация данных

Теперь займёмся внешними преобразованиями, чтобы в дальнейшем было проще отвечать на вопросы заказчика

Сначала добавим столбец *purpose_name* выбрав данные по 4 основным категориям(свадьба, автомобиль, недвижимость, образование)

In [ ]:
print(clients['purpose_lemma'].value_counts()) #ещё раз убедимся, что взяли правильные основные категории

In [ ]:
def replace_purpose(i):
    if 'свадьба' in i:
        return 'свадьба'
    elif 'автомобиль' in i:
        return 'авто'
    elif 'недвижимость' in i or 'жилье' in i:
        return 'недвижимость'
    elif 'образование' in i:
        return 'образование'
    else:
        return 5
    
clients ['purpose_name'] = clients ['purpose_lemma'].apply(replace_purpose)
clients ['purpose_name'].value_counts()

In [ ]:
clients.head(15)

Дальше разделим *total_income* по квантилям и добавим новый столбец разделив уровень дохода на 4 группы (Низкий, Средний, Высокий и Сверхвысокий)

In [ ]:
clients['total_income'].quantile([0.25,0.5,0.75])

In [ ]:
def income_level(total_income):
    if total_income <= 107623.00:
            return 'Низкий'
    if total_income <= 156043.50:
            return 'Средний'
    if total_income < 195813.25:
            return 'Высокий'
    return 'Сверхвысокий'

clients['income_level'] = clients['total_income'].apply(income_level)
clients.head(15)

**Вывод**

Мы провели категоризацию в соотвествии с леммами в столбце puprpose_lemma выявив 4 освновные категории для целей кредита. А также добавили столбец с разделением уровней дохода на 4 группы. Всё это понадобится нам в дальнейшем при ответе на вопросы связанными с целями кредита и уровнем дохода.

Категорировать количество детей нет необходимости, т.к. категориями будет выступать их количество (мы имеем всего 6 вариантов - от 0 до 5).

## Шаг 3. Ответьте на вопросы

***- Есть ли зависимость между наличием детей и возвратом кредита в срок?***

Построим новую таблицу методом DataFrame()

In [ ]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = clients.groupby('children')['debt'].count()
debt_from_children['sum_children'] = clients.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] * 100
debt_from_children

Отсортируем таблицу по убыванию относительных значений для пущей наглядности

In [ ]:
debt_from_children.sort_values('result_children', ascending = False)

**Вывод**

Как видно, данные не дают нам однозначных выводов по поводу влияния количества детей на возврат кредита в срок. Однако, исходя из них можно предположить, что люди не имеющие детей чаще платят в срок(скорее всего из за того, что имеют меньше расходов - отсутствуют расходы на детей). При этом люди имеющие 3 детей имеют лучшую кредитную историю, чем люди с 1 или 2 детьми(возможно причина как раз и кроется в кредитной истории - она должны быть хорошей, чтобы в дальнейшем им одобрили новые кредиты).

***- Есть ли зависимость между семейным положением и возвратом кредита в срок?***

Проделываем то же, что и при ответе на предыдущий вопрос, с той лишь разницей, что сразу отсортировываем значения по убыванию, чтобы увидеть закономерности, которые нас инетресуют

In [ ]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['count_family_status'] = clients.groupby('family_status')['debt'].count()
debt_from_family_status['sum_family_status'] = clients.groupby('family_status')['debt'].sum()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] * 100
debt_from_family_status.sort_values('result_family_status', ascending = False)

**Вывод**

Мы видим интересную зависимость: овдовевшие и разведённые чаще возвращают кредит в срок. Люди не бывавшие в браке наоборот чаще просрачивают платежи по кредитам. Могу предположить, что люди прошедшие через брак просто располагают большим жизненным опытом и потому лучше знакомы с принципами финансовой грамотности и планированием бюджета

***- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?***

Тут мы сталкиваемся с необходимостью сравнить уровни дохода клиентов банка и разложить по полочкам должников по кредитам. В этом нам поможет разбивка по уровню дохода, которую мы проделывали в пункте *2.5. Категоризация данных*, а также сплав методов использованных в предыдущих ответах на вопросы с методами построения сводных таблиц

In [ ]:
total_income_set = clients.groupby('income_level')['debt'].sum() / clients.groupby('income_level')['debt'].count() * 100
total_income_set.sort_values()

In [ ]:
total_income_pivot = clients.pivot_table(index=['income_level'], columns=['debt'], values='education_id', aggfunc='count')
total_income_pivot['rating'] = total_income_pivot[1] / total_income_pivot[0] * 100
total_income_pivot

Почему то некоторые значения потерялись, но тенденции остались прежними(а заключаются они в том, что если ты много зарабатываешь, то можешь спокойно брать кредит, т.к. знаешь что проблем с его возратом не возникнет. Всем бы так)

**Вывод**

Зависимость если и есть, то она вполне закономерна: Люди со сверхвысокими доходами лучше отдают кредиты, потому что имеют такую возможность. А люди с низкими доходами вынуждены в срок расплачиваться по кредитам, чтобы не загубить кредитную историю для будущих займов. Не думаю, что на основании только этих данных, можно строить качественные прогнозы

***- Как разные цели кредита влияют на его возврат в срок?***

Тут мы также пожинаем плоды категоризации, проделанной в пункте *2.5.*, а именно имеем возможность отсортировать возврат кредитов по категориям, которые мы выделили для целей кредита

In [ ]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = clients.groupby('purpose_name')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = clients.groupby('purpose_name')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category'] * 100 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

**Вывод**

Люди, берущие кредиты на недвижимость, более ответственно подходят к выплатам. Причиной этому может быть осознание людей, что своя крыша над головой - это важно и нужно. Также люди с собственной жил. площадью избавлены от выплат за аренду жилого помещения(или удачно гасят эти выплаты сдавая в аренду свою собственность). А вот с кредитами на автомобили всё не так радужно. То ли автолюбители менее ответственны в плане финансов, то ли цены на бензин толкают их на нарушение сроков выплат. Чтобы не послужило причиной, закономерность имеется и с ней нужно считаться

## Шаг 4. Общий вывод

Отвечая на поставленный в самом начале проекта вопрос **Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок**, можно сказать - **да, влияют.**

*Исходя из проделанной работы, можно составить оптимальный образ заёмщика:* это не имеющий детей, находящийся или ранее находившийся в браке человек (с низким или сверх высоким уровнем дохода, берущий кредит на жильё).

В то же время, люди, состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей - самые менее ответственные заемщики (особенно если у них средние урони дохода, а кредит они берут на свадьбу или обучение)